# Hitmaps and White noise covariance
> Load and scale hitmaps and white noise covariance matrices
- toc: True

* Basically the same scaling of the noise

In [ ]:
# default_exp hitmap_wcov

In [ ]:
import logging as log

In [ ]:
# export

import healpy as hp
import numpy as np
from pathlib import Path
import logging as log

from s4_design_sim_tool.core import get_telescope, base_folder, simulations_observing_efficiency
from s4_design_sim_tool.noise import get_thinfp, get_tube_years

In [ ]:
# exports


def load_hitmap_wcov(config, site, channel, realization=0):
    """Load hitmaps and white noise covariance for a channel

    Parameters
    ----------
    config : dict
        CMB-S4 configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    realization : int
        Choose one of the available 8 realizations

    Returns
    -------
    output_map : numpy array
        Output map with all emissions combined, uses nan for missing pixels
    """
    
    # it is the same scaling for hitmap and white noise covariance matrix,
    # which is the same as noise except squared
    
    telescope = get_telescope(channel)
    
    scaling = (365.25 * config["experiment"]["observing_efficiency"]) / (
        10
        * simulations_observing_efficiency[site.lower()].get(telescope, 1)
    )
    scaling *= get_tube_years(config, site, channel)
    # focal plane thinning factor of TOD simulations
    scaling *= get_thinfp(channel)
    
    map_filename = (
        Path(f"{realization:08d}")
        / f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_hmap.fits"
    )
    wcov_filename = (
        Path(f"{realization:08d}")
        / f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_wcov.fits"
    )
    log.info(f"Base folder: {base_folder}")
    log.info(f"Reading {map_filename}")
    hitmap = hp.read_map(
        Path(base_folder) / map_filename, 0, dtype=None, verbose=False
    )
    wcov = hp.read_map(
        Path(base_folder) / wcov_filename, (0,1,2,3,4,5), dtype=None, verbose=False
    )
    hitmap[hitmap == hp.UNSEEN] = 0
    # input map is 10 days at 100% efficiency
    hitmap = np.round(hitmap * scaling).astype(np.int64)
    wcov /= scaling
    wcov[:, hitmap==0] = hp.UNSEEN
    return hitmap, wcov

## Available input maps

In [ ]:
import toml

config = toml.load("s4_reference_design.toml")

In [ ]:
filenames = !ls $base_folder/00000000/*noise*wcov*

In [ ]:
import os.path
for f in map(os.path.basename, filenames):
    print(f)

In [ ]:
channel = "HFS1"
site = "Pole"

In [ ]:
telescope = get_telescope(channel)

In [ ]:
input_map = hp.read_map(
    base_folder + "/00000000/" + \
    f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_hmap.fits"
, 0)

In [ ]:
from astropy.io import fits

In [ ]:
fits.open(base_folder + "/00000000/" + \
    f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_wcov.fits"
)[1].header

In [ ]:
hitmap, wcov = load_hitmap_wcov(config, site, channel, realization=0)

In [ ]:
num_tubes = sum([
    telescope[channel[:-1]] for telescope in config["telescopes"][telescope].values()
])

In [ ]:
sampling_rate = hitmap.sum() / (5 * config["experiment"]["observing_efficiency"])
sampling_rate /= 365.25 * 24 * 3600
sampling_rate /= 8438 # HF detectors per tube
sampling_rate /= num_tubes

In [ ]:
sampling_rate

In [ ]:
# Compare mask
np.testing.assert_allclose(input_map == 0, hitmap == 0)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
log_hitmap = np.log10(hitmap)

In [ ]:
log_hitmap[np.isinf(log_hitmap)] = hp.UNSEEN

In [ ]:
hp.mollview(log_hitmap, unit="log10(samples)", min=3, max=8.5, title="Hitmap")

In [ ]:
for i,label in [(3, "EE"), (4, "EB"), (5, "BB")]:
    hp.mollview(wcov[i], unit="$K^2_{CMB}$", min=0, max=1e-11, title=f"{label} White noise covariance")